This script runs a simulation of two interacting colonies with multiple chemotactic waves. It does this computationally by running 2 PDE simulations. The first only allows one chemotactic wave to expand, the second is held fixed. Then, after running for some set time, the result of that simulation is used as the initial conditions for the second simulation that allows both chemotactic waves to expand.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx, Ly = (24, 20)
nx, ny = (256, 256)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun_a(A,B):
    F = (A.data)*(1-4*B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator_a(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun_a,
        args = (field1,field2,)
    )

de.operators.parseables['grow_a'] = grow_operator_a

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1- (4/3)*B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rho12'])
problem.parameters['D'] = Diff
problem.parameters['K'] = K
problem.parameters['g'] = g

problem.substitutions['rho_t'] = "rho1+rho2+rho12" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem.add_equation("dt(rho1)  - D*Lap(rho1) = g*grow_a(rho1,rho_t)  - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D*Lap(rho2) = g*grow_a(rho2,rho_t)  - K*rho1*rho2")

problem.add_equation("dt(rho12)  = 2*K*rho1*rho2")

In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']


def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

r0 = 0.75
w = 0.22

rho1['g'] = 0.25*blob(x,y,Lx*0.5 - 4.5,0.5*Ly,r0,w)  
rho2['g'] = 0.25*blob(x,y,Lx*0.5 + 4.5,0.5*Ly,r0,w)

In [ ]:
solver.stop_sim_time = 100
dt = 0.25*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('interface', sim_dt=1, max_writes=500)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho12')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

The first simulation has finished running, now we start the second one

In [ ]:
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

problem_part2 = de.IVP(domain, variables=['rho1a','rho1b','rho2a','rho2b','rho12'])
problem_part2.parameters['D'] = Diff
problem_part2.parameters['K'] = K
problem_part2.parameters['ga'] = g
problem_part2.parameters['gb'] = g

problem_part2.substitutions['rho1'] = "rho1a+rho1b" 
problem_part2.substitutions['rho2'] = "rho2a+rho2b" 
problem_part2.substitutions['rho_ta'] = "rho1a+rho2a+rho12" 
problem_part2.substitutions['rho_tb'] = "rho1b+rho2b+rho12" 
problem_part2.substitutions['rho_t'] = "rho1+rho2+rho12" 
problem_part2.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem_part2.add_equation("dt(rho1a)  - D*Lap(rho1a) = ga*grow_a(rho1a,rho_ta)  - K*rho1a*rho2")
problem_part2.add_equation("dt(rho1b)  - D*Lap(rho1b) = gb*grow(rho1b,rho_tb)  - K*rho1b*rho2")

problem_part2.add_equation("dt(rho2a)  - D*Lap(rho2a) = ga*grow_a(rho2a,rho_ta)  - K*rho1*rho2a")
problem_part2.add_equation("dt(rho2b)  - D*Lap(rho2b) = gb*grow(rho2b,rho_tb)  - K*rho1*rho2b")

problem_part2.add_equation("dt(rho12)  = 2*K*rho1*rho2")

In [ ]:
solver_part2 =  problem_part2.build_solver(ts)

In [ ]:
from dedalus.tools import post
post.merge_process_files("interface", cleanup=True)

In [ ]:
f = h5py.File('interface/interface_s1.h5','r')
y = f['/scales/y/1.0'][:]
x = f['/scales/x/1.0'][:]
t = f['scales']['sim_time'][:]
rho1 = f['tasks']['rho1'][:]
rho2 = f['tasks']['rho2'][:]
rhom = f['tasks']['rho12'][:]

rho_t = rho1 + rho2 + rhom

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1a = solver_part2.state['rho1a']
rho1b = solver_part2.state['rho1b']
rho2a = solver_part2.state['rho2a']
rho2b = solver_part2.state['rho2b']

r0 = 0.75
w = 0.22

rho1a['g'] = rho1[-1,:,:] 
rho1b['g'] = 0.75*blob(x,y,Lx*0.5 - 4.5,0.5*Ly,r0,w)  

rho2a['g'] = rho2[-1,:,:] 
rho2b['g'] = 0.75*blob(x,y,Lx*0.5 + 4.5,0.5*Ly,r0,w)

In [ ]:
solver_part2.stop_sim_time = 300
dt = 0.25*Lx/nx

In [ ]:
analysis_part2 = solver_part2.evaluator.add_file_handler('interface_part2', sim_dt=1, max_writes=500)
analysis_part2.add_task('rho1a')
analysis_part2.add_task('rho1b')
analysis_part2.add_task('rho2a')
analysis_part2.add_task('rho2b')
analysis_part2.add_task('rho12')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver_part2.ok:
    solver_part2.step(dt)
    if solver_part2.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver_part2.iteration, solver_part2.sim_time, dt))

In [ ]:
from dedalus.tools import post
post.merge_process_files("interface_part2", cleanup=True)